# Sobol Sensitivity Indices for the BTD Model

## Setup packages.

In [1]:
require(data.table)
require(magrittr)
require(sensitivity)
require(SobolSequence)

require(ggplot2)

Loading required package: data.table
Loading required package: magrittr
Loading required package: sensitivity
Registered S3 method overwritten by 'sensitivity':
  method    from 
  print.src dplyr
Loading required package: SobolSequence
Loading required package: ggplot2


## Design experiment.

### Load input ranges.

In [2]:
z.ranges <- fread("input-ranges-si.tsv")
z.ranges %>% dim

[1] 32  9

In [3]:
z.ranges %>% summary

   Variable            Units               Type           Model Minimum   
 Length:32          Length:32          Length:32          Min.   :  0.00  
 Class :character   Class :character   Class :character   1st Qu.:  0.00  
 Mode  :character   Mode  :character   Mode  :character   Median :  0.01  
                                                          Mean   : 17.89  
                                                          3rd Qu.:  1.00  
                                                          Max.   :168.00  
 Model Maximum      Model Default       Sensitivity Minimum Sensitivity Maximum
 Length:32          Min.   :        0   Min.   :        0   Min.   :0.000e+00  
 Class :character   1st Qu.:        0   1st Qu.:        0   1st Qu.:3.000e+00  
 Mode  :character   Median :        1   Median :        0   Median :1.400e+01  
                    Mean   : 12547769   Mean   :  3764340   Mean   :5.865e+07  
                    3rd Qu.:     1664   3rd Qu.:      512   3rd Qu.:1.058e+

### Experiment based on Sobol sequences.

In [4]:
z.design <- sobolSequence.points(dim(z.ranges)[1], count = 10000)
colnames(z.design) <- z.ranges$Variable
z.design %>% dim

[1] 10000    32

In [5]:
write.table(z.design, file = "design-si.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

### Relate the design to the model's variables.

In [6]:
z.inputs <- z.design
for (i in 1:dim(z.ranges)[1]) {
    z.inputs[, i] <- z.ranges[i, `Sensitivity Minimum`] * (1 - z.inputs[, i]) +
                     z.ranges[i, `Sensitivity Maximum`] *      z.inputs[, i]
    if (z.ranges[i, `Type`] %in% c("Integer", "Boolean"))
        z.inputs[, i] <- round(z.inputs[, i])
}
z.inputs <- data.table(
    Run = 1:dim(z.inputs)[1],
    z.inputs
)
z.inputs %>% summary

      Run        aversion to NPV deviation base external investor ask rate
 Min.   :    1   Min.   :0.0600            Min.   : 2.4                   
 1st Qu.: 2501   1st Qu.:0.7099            1st Qu.: 6.4                   
 Median : 5000   Median :1.3598            Median :10.4                   
 Mean   : 5000   Mean   :1.3598            Mean   :10.4                   
 3rd Qu.: 7500   3rd Qu.:2.0098            3rd Qu.:14.4                   
 Max.   :10000   Max.   :2.6597            Max.   :18.4                   
 bioproduct long term price bioproduct performance advantage
 Min.   :1500               Min.   :0.350                   
 1st Qu.:3375               1st Qu.:1.712                   
 Median :5250               Median :3.075                   
 Mean   :5250               Mean   :3.075                   
 3rd Qu.:7124               3rd Qu.:4.437                   
 Max.   :8999               Max.   :5.800                   
 commercial capital cost input commercial plant 

In [7]:
write.table(z.inputs, file="inputs-si.tsv", row.names = FALSE, col.names = TRUE, sep = "\t", quote = FALSE)

## Analyze results.

### Read files.

#### Read design.

In [8]:
z.design <- fread("design-si.tsv")
z.design %>% dim

[1] 10000    32

#### Read inputs.

In [9]:
z.inputs <- fread("inputs-si.tsv")
z.inputs %>% dim

[1] 10000    33

#### Read outputs.

In [10]:
z.outputs <- fread("outputs-si.tsv")
z.outputs[Time == 2050] %>% dim

ERROR: Error in fread("outputs-si.tsv"): File 'outputs-si.tsv' does not exist or is non-readable. getwd()=='/home/bbush/Projects/Active/bioproducts/BioproductTransitionDynamics/sensitivity'


In [ ]:
z.outputs[Time == 2050] %>% summary

### Compute sensitivity indices.

#### Just use the final year.

In [ ]:
z.outputs.clean <- z.outputs[order(Run)][`Time` == 2050, c(-1, -2)] %>% as.matrix
z.outputs.clean %>% dim